In [75]:
from langchain.document_loaders import DirectoryLoader,TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain.chat_models import init_chat_model
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os
from dotenv import load_dotenv

In [76]:
root_path =r"C:\Users\Mohamed Arshad\Downloads\My_RAG_Lab\llm_engineering\RAG\knowledge-base"

In [77]:
from langchain.document_loaders import DirectoryLoader,TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain.chat_models import init_chat_model
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os
from dotenv import load_dotenv

In [78]:
root_path =r"C:\Users\Mohamed Arshad\Downloads\My_RAG_Lab\llm_engineering\RAG\knowledge-base"

In [79]:
loader = DirectoryLoader(path=root_path,
                         glob="**/*.md",
                         loader_cls=TextLoader,
                         loader_kwargs={"encoding":"utf-8"})

try:
    docs =loader.load()
    print(f"Document loaded with {len(docs)} from {root_path}")

except Exception as e:
    print(f'Error occured {e}')


Document loaded with 76 from C:\Users\Mohamed Arshad\Downloads\My_RAG_Lab\llm_engineering\RAG\knowledge-base


In [80]:
docs[60]

Document(metadata={'source': 'C:\\Users\\Mohamed Arshad\\Downloads\\My_RAG_Lab\\llm_engineering\\RAG\\knowledge-base\\employees\\Oliver Spencer.md'}, page_content='# HR Record\n\n# Oliver Spencer\n\n## Summary\n- **Date of Birth**: May 14, 1990\n- **Job Title**: Backend Software Engineer\n- **Location**: Austin, Texas\n- **Current Salary**: $125,000  \n\n## Insurellm Career Progression\n- **March 2018**: Joined Insurellm as a Backend Developer I, focusing on API development for customer management systems.\n- **July 2019**: Promoted to Backend Developer II after successfully leading a team project to revamp the claims processing system, reducing response time by 30%.\n- **June 2021**: Transitioned to Backend Software Engineer with a broader role in architecture and system design, collaborating closely with the DevOps team.\n- **September 2022**: Assigned as the lead engineer for the new "Innovate" initiative, aimed at integrating AI-driven solutions into existing products.\n- **January

Initialize Hybrid Retriever ,then go for Reranker 

In [81]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
chunks =text_splitter.split_documents(documents=docs)

In [82]:
embedding_model =HuggingFaceEmbeddings(model="all-MiniLM-L6-V2")
dense_vector_store = Chroma.from_documents(documents=chunks,embedding=embedding_model)
dense_retreiver =dense_vector_store.as_retriever(search_kwargs={"k":"5"})

In [83]:
load_dotenv(override=True)

True

In [84]:
groq_api_key = os.getenv("GROQ_API_KEY")
llm =ChatGroq(model="gemma2-9b-it")

In [85]:
loader = DirectoryLoader(path=root_path,
                         glob="**/*.md",
                         loader_cls=TextLoader,
                         loader_kwargs={"encoding":"utf-8"})

try:
    docs =loader.load()
    print(f"Document loaded with {len(docs)} from {root_path}")

except Exception as e:
    print(f'Error occured {e}')


Document loaded with 76 from C:\Users\Mohamed Arshad\Downloads\My_RAG_Lab\llm_engineering\RAG\knowledge-base


In [86]:
docs[65]

Document(metadata={'source': 'C:\\Users\\Mohamed Arshad\\Downloads\\My_RAG_Lab\\llm_engineering\\RAG\\knowledge-base\\employees\\Samuel Trenton.md'}, page_content='# HR Record\n\n# Samuel Trenton\n\n## Summary\n- **Date of Birth:** April 12, 1989\n- **Job Title:** Senior Data Scientist\n- **Location:** Austin, Texas\n- **Current Salary:** $115,000  \n\n## Insurellm Career Progression\n- **January 2020 - Present:** Senior Data Scientist  \n  *Promoted for demonstrating exceptional analytical skills and leadership potential. Led several projects that improved customer segmentation strategies, resulting in a 15% increase in customer retention.*\n\n- **June 2018 - December 2019:** Data Scientist  \n  *Joined the Insurellm team and worked on developing predictive modeling techniques to assess risk for both B2B and B2C customers. Received recognition for the success of the "Risk Assessment Model" project.*\n\n- **August 2016 - May 2018:** Junior Data Analyst  \n  *Started at Insurellm as a J

Initialize Hybrid Retriever ,then go for Reranker 

In [87]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
chunks =text_splitter.split_documents(documents=docs)

 #### as_retriever() capabilities in Vector store

- as_retriever()
  • Default retriever  
  • Returns top 4 most similar chunks

- as_retriever(search_kwargs={"k": 10})
  • Returns top 10 most similar chunks  
  • "k" = number of results

- as_retriever(search_type="mmr")
  • MMR = Maximal Marginal Relevance  
  • Retrieves diverse + relevant chunks  
  • Reduces redundancy

- as_retriever(search_kwargs={"score_threshold": 0.5})
  • Only return chunks with similarity score ≥ 0.5  
  • Helps remove weak/irrelevant chunks

- as_retriever(search_kwargs={"filter": {"source": "icsr"}})
  • Metadata-based filtering  
  • Only returns chunks matching metadata (example: source="icsr")

- as_retriever(search_type="similarity", search_kwargs={"k": 5, "filter": {...}})
  • Combine multiple options  
  • Choose similarity-based search + limit results + filter metadata

- as_retriever(search_type="mmr", search_kwargs={"k": 8, "lambda_mult": 0.3})
  • MMR with custom diversity level  
  • lambda_mult controls relevance vs diversity

- as_retriever(search_kwargs={"where_document": {"$contains": "AE"}})
  • Filter based on document content (Chroma text filter)



In [88]:
embedding_model =HuggingFaceEmbeddings(model="all-MiniLM-L6-V2")
dense_vector_store = Chroma.from_documents(documents=chunks,embedding=embedding_model)
dense_retreiver =dense_vector_store.as_retriever(search_kwargs={"k":5})

Use groq llm for Reranking (Use smaller models here )

In [89]:
groq_api_key = os.getenv("GROQ_API_KEY")
llm =ChatGroq(model="llama-3.1-8b-instant")

In [90]:
# Prompt Template
prompt = PromptTemplate.from_template("""
You are a helpful assistant. Your task is to rank the following documents from most
to least relevant to the user's question.

User Question: {question}

Documents:
{documents}

Instructions:
- Think about the relevance of each document to the user's question.
- Return a list of document indices in ranked order, starting from the most relevant.

Output format: comma-separated document indices (e.g., 2,1,3,0,...)
""")

In [91]:
query ="who got prestigious IIOTY award in 2023?"

In [92]:
retreived_docs=dense_retreiver.invoke(query)
retreived_docs

[Document(id='f2ddc1aa-0888-4216-9ff4-31773e6fd70a', metadata={'source': 'C:\\Users\\Mohamed Arshad\\Downloads\\My_RAG_Lab\\llm_engineering\\RAG\\knowledge-base\\employees\\Robert Chen.md'}, page_content="## Other HR Notes\n- **Education:** MS in Computer Science from Stanford University, BS in Computer Engineering from MIT\n- **Technical Expertise:** Expert in React, Node.js, TypeScript, PostgreSQL, AWS, microservices architecture\n- **Recognition:** Engineering Excellence Award 2023, Technical Leadership Award 2021\n- **Mentorship:** Mentored 12 engineers during tenure. Known for developing junior talent into strong contributors.\n- **Patents:** Co-inventor on 1 patent for insurance processing optimization\n- **Feedback:** World-class engineer with exceptional technical and leadership skills. Trusted advisor to engineering leadership. Critical to company's technical success."),
 Document(id='21d32b8b-6d21-44ca-8dba-654c3f08f5e2', metadata={'source': 'C:\\Users\\Mohamed Arshad\\Downlo

In [93]:
for i,doc in enumerate(retreived_docs):
    print(f"Source:\n{doc.metadata.get("source","Unknown source")}")
    print(f"{i+1} {doc.page_content}\n")

Source:
C:\Users\Mohamed Arshad\Downloads\My_RAG_Lab\llm_engineering\RAG\knowledge-base\employees\Robert Chen.md
1 ## Other HR Notes
- **Education:** MS in Computer Science from Stanford University, BS in Computer Engineering from MIT
- **Technical Expertise:** Expert in React, Node.js, TypeScript, PostgreSQL, AWS, microservices architecture
- **Recognition:** Engineering Excellence Award 2023, Technical Leadership Award 2021
- **Mentorship:** Mentored 12 engineers during tenure. Known for developing junior talent into strong contributors.
- **Patents:** Co-inventor on 1 patent for insurance processing optimization
- **Feedback:** World-class engineer with exceptional technical and leadership skills. Trusted advisor to engineering leadership. Critical to company's technical success.

Source:
C:\Users\Mohamed Arshad\Downloads\My_RAG_Lab\llm_engineering\RAG\knowledge-base\employees\Robert Chen.md
2 ## Other HR Notes
- **Education:** MS in Computer Science from Stanford University, BS in 

In [94]:
chain =prompt|llm|StrOutputParser
chain

PromptTemplate(input_variables=['documents', 'question'], input_types={}, partial_variables={}, template="\nYou are a helpful assistant. Your task is to rank the following documents from most\nto least relevant to the user's question.\n\nUser Question: {question}\n\nDocuments:\n{documents}\n\nInstructions:\n- Think about the relevance of each document to the user's question.\n- Return a list of document indices in ranked order, starting from the most relevant.\n\nOutput format: comma-separated document indices (e.g., 2,1,3,0,...)\n")
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002512329F050>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002512329F1A0>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))
| RunnableLambda(StrOutputParser)

In [95]:
doc_lines = [f"{i+1}. {doc.page_content}" for i, doc in enumerate(retreived_docs)]
formatted_docs = "\n".join(doc_lines)

In [100]:
from langchain_core.runnables import RunnableLambda

In [101]:
# Create runnable chain
parse_output = RunnableLambda(lambda x: x)  # Just return string
chain = prompt | llm | parse_output

# Run chain
response = chain.invoke({"question": query, "documents": formatted_docs})
print("Reranked indices:", response)

Reranked indices: content="Based on the user's question, the most relevant document would be the one that contains information about the IIOTY (Industry Innovation of the Year) award in 2023. However, none of the documents provided explicitly mention the IIOTY award. \n\nBut we can infer that the documents are HR notes and performance history, and they do not contain any information about industry-wide awards like IIOTY. The documents do mention an Engineering Excellence Award 2023, but it's not the same as IIOTY.\n\nGiven the information available, I would rank the documents as follows based on the relevance to the user's question:\n\nThe ranking is based on the fact that none of the documents directly answer the question. However, the documents with more details or information are ranked higher.\n\nThe most relevant documents are likely to be ranked lower as they don't contain any information about the IIOTY award.\n\nThe least relevant documents are likely to be ranked higher as the